In [1]:
import numpy as np
import sys
if "../" not in sys.path:
    sys.path.append("../")
from lib.envs.gridworld import GridworldEnv

In [48]:
env = GridworldEnv(shape=[4,4])
env._render()

T  o  o  o
o  o  o  o
x  o  o  o
o  o  o  T


In [49]:
def policy_evaluation(policy, env, V=np.zeros(env.nS), discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    it = 0
    while True:
        # TODO: Implement!
        delta = 0
        for s in range(env.nS):
            v = 0
            # possible next actions
            for a, action_prob in enumerate(policy[s]):
                # possible next states
                for (prob, next_state, reward, done) in env.P[s][a]:
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            delta = np.maximum(delta, abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break
        
        if (it+1) % 10 == 0:
            print('Iteration: {}'.format(it + 1))
            print('Delta: {}'.format(delta))
        it += 1
        
    return np.array(V)

In [50]:
def policy_iteration(env, policy_eval_fn, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
    """
    # start with a random policy
    # TODO: create random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    V = policy_eval_fn(policy, env)
    
    # implement one step lookahead to calculate value of all actions in a given state
    def one_step_lookahead(state, V):
        """
        Helper fn to calculate value of all actions in a given state
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros((env.nA))
        for a in range(env.nA):
            for (prob, nex_state, reward, done) in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[nex_state])
        return A
    
    while True:
        V = policy_eval_fn(policy, env, V, discount_factor)
        policy_stable = True
        
        for s in range(env.nS):
            # get the best action
            old_action = np.argmax(policy[s])
            
            # one step lookahead and get the best action to take
            action_values = one_step_lookahead(s, V)
            best_action = np.argmax(action_values)
            if old_action != best_action:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_action]
        if policy_stable:
            break
        
    return policy, V

In [51]:
policy, V = policy_iteration(env, policy_evaluation)

Iteration: 10
Delta: 0.8924059502996897
Iteration: 20
Delta: 0.372594031842965
Iteration: 30
Delta: 0.1552741675450946
Iteration: 40
Delta: 0.06470839506731352
Iteration: 50
Delta: 0.02696634228047401
Iteration: 60
Delta: 0.011237855848747813
Iteration: 70
Delta: 0.004683223359091215
Iteration: 80
Delta: 0.0019516695467842737
Iteration: 90
Delta: 0.0008133317007938956
Iteration: 100
Delta: 0.0003389449082717988
Iteration: 110
Delta: 0.0001412506739058017
Iteration: 120
Delta: 5.886429443435759e-05
Iteration: 130
Delta: 2.453089294007782e-05
Iteration: 140
Delta: 1.0222915509672248e-05


In [54]:
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print()

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]



In [56]:
print("Value Function: ")
print(V)
print("")

print("Reshape value function according to grid: ")
print(np.reshape(V, env.shape))

Value Function: 
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshape value function according to grid: 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
